# Wide & Deep Learning model 구현

* 라이브러리 불러오기

In [56]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

In [57]:
df3_merge = pd.read_csv('./data/df3_merge.csv')
df3_merge = df3_merge.sort_values(['sess_dt', 'hit_tm'])
df3_merge.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,latest_act_hr_1,latest_act_hr_2,latest_act_hr_3,hum,temp,pty,r06,clnt_gender,clnt_age,clac_nm2
121082,3390,1,1,0,A01,2019-07-01,00:00,0,NaN,지고트,...,-1.0,-1.0,-1.0,59.0,24.700001,0.0,0.0,NaN,NaN,NaN
194580,5535,1,1,5,A03,2019-07-01,00:00,6532,NaN,NaN,...,-1.0,-1.0,-1.0,59.0,24.700001,0.0,0.0,F,30.0,NaN
194581,5535,1,2,0,A03,2019-07-01,00:00,30494,NaN,양파,...,-1.0,-1.0,-1.0,59.0,24.700001,0.0,0.0,F,30.0,NaN
194582,5535,1,3,3,A03,2019-07-01,00:00,32370,NaN,NaN,...,-1.0,-1.0,-1.0,59.0,24.700001,0.0,0.0,F,30.0,NaN
194583,5535,1,4,0,A03,2019-07-01,00:00,41637,NaN,우엉,...,-1.0,-1.0,-1.0,59.0,24.700001,0.0,0.0,F,30.0,NaN


In [58]:
print('구매 이력이 있는 행:', round(df3_merge['clac_nm2'].notna().sum() / df3_merge['clac_nm2'].isna().sum() * 100, 2),'%')

구매 이력이 있는 행: 3.88 %


In [59]:
user2vec = pd.read_csv('./data/user2vec.csv')
df_buyer = df3_merge[df3_merge['clac_nm2'].notna()]
df_buyer = df_buyer.merge(user2vec, on=['clnt_id', 'sess_id'], how='left')

In [60]:
drop_col=['clnt_id', 'sess_id', 'hit_seq', 'action_type', 'hit_tm', 'trans_id', 'sech_kwd']
df_buyer = df_buyer.drop(drop_col, axis=1)

In [61]:
le = LabelEncoder()

lab_len = len(df_buyer['clac_nm2'].value_counts())
label_key = df_buyer['clac_nm2'].value_counts().keys()

df_buyer['clac_nm2'] = le.fit_transform(df_buyer['clac_nm2'])
clac_nm2 = df_buyer['clac_nm2'].copy()
df_buyer.drop(['clac_nm2'], axis=1, inplace=True)

In [62]:
df_buyer.loc[df_buyer['pv_hr'].isnull(),'pv_hr'] = 0
df_buyer.loc[df_buyer['latest_pv_hr_1'].isnull(),'latest_pv_hr_1'] = 0
df_buyer.loc[df_buyer['latest_pv_hr_2'].isnull(),'latest_pv_hr_2'] = 0
df_buyer.loc[df_buyer['latest_pv_hr_3'].isnull(),'latest_pv_hr_3'] = 0

In [63]:
data = df_buyer.copy()

## 데이터 전처리

In [64]:
ALL_CATEGORICAL_COLUMNS = [
    "biz_unit", "sess_dt", "tot_pag_view_ct", 'trfc_src', 'trfc_src', 'dvc_ctg_nm',
    'cum_act_0', 'cum_act_1', 'cum_act_2', 'cum_act_3', 'cum_act_4', 'cum_act_5', 'cum_act_6',
    'cum_act_7', 'day', 'holiday', 'hour',  'prefer_dvc_trfc', 'clnt_gender',
    'clnt_age', 'pty', 'r06', 'latest_act_hr_1', 'latest_act_hr_2', 'latest_act_hr_3',
    'X_0', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6',
       'X_7', 'X_8', 'X_9', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15',
       'X_16', 'X_17', 'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24',
       'X_25', 'X_26', 'X_27', 'X_28', 'X_29'
]

CATEGORICAL_COLUMNS = [
    "biz_unit", "sess_dt", "tot_pag_view_ct", 'trfc_src', 'trfc_src', 'dvc_ctg_nm',
    'cum_act_0', 'cum_act_1', 'cum_act_2', 'cum_act_3', 'cum_act_4', 'cum_act_5', 'cum_act_6',
    'cum_act_7', 'day', 'holiday', 'hour',  'prefer_dvc_trfc', 'clnt_gender',
    'clnt_age', 'pty', 'r06', 'latest_act_hr_1', 'latest_act_hr_2', 'latest_act_hr_3'
]

EMBEDDED_COLUMNS = [
    'X_0', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6',
       'X_7', 'X_8', 'X_9', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15',
       'X_16', 'X_17', 'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24',
       'X_25', 'X_26', 'X_27', 'X_28', 'X_29'
]

CONTINUOUS_COLUMNS = ['latest_pv_hr_3','latest_pv_hr_2', 'hit_pss_tm', 'pv_hr', 'tot_sess_hr_v', 'latest_pv_hr_1', 'temp', 'hum']

In [65]:
for c in CATEGORICAL_COLUMNS:
    le = LabelEncoder()
    data[c] = le.fit_transform(data[c])

## Train 데이터, Test 데이터 분할

In [66]:
label = clac_nm2.copy()
label = np.eye(lab_len)[label]
label = pd.DataFrame(data=label, columns=label_key, index=df_buyer.index)

In [67]:
train_x, test_x , train_y , test_y = train_test_split(data , label , test_size=0.05, shuffle=False)

In [68]:
print('Train 데이터: ', train_x.shape)
print('Test 데이터: ', test_x.shape)
print('Train 라벨: ', train_y.shape)
print('Test 라벨: ', test_y.shape)

Train 데이터:  (116940, 62)
Test 데이터:  (6155, 62)
Train 라벨:  (116940, 288)
Test 라벨:  (6155, 288)


In [69]:
train_x, val_x , train_y , val_y = train_test_split(train_x , train_y , test_size=0.1, shuffle=False)

In [70]:
print('Train 데이터: ', train_x.shape)
print('Val 데이터: ', val_x.shape)
print('Train 라벨: ', train_y.shape)
print('Val 라벨: ', val_y.shape)

Train 데이터:  (105246, 62)
Val 데이터:  (11694, 62)
Train 라벨:  (105246, 288)
Val 라벨:  (11694, 288)


## 카테고리 값들과 연속값들을 뽑아냄

In [71]:
train_x_category = np.array(train_x[CATEGORICAL_COLUMNS])
test_x_category  = np.array(test_x[CATEGORICAL_COLUMNS])
val_x_category   = np.array(val_x[CATEGORICAL_COLUMNS])

train_x_embedding = np.array(train_x[EMBEDDED_COLUMNS])
test_x_embedding  = np.array(test_x[EMBEDDED_COLUMNS])
val_x_embedding   = np.array(val_x[EMBEDDED_COLUMNS])

train_x_continue = np.array(train_x[CONTINUOUS_COLUMNS], dtype='float64')
test_x_continue = np.array(test_x[CONTINUOUS_COLUMNS], dtype='float64')
val_x_continue = np.array(val_x[CONTINUOUS_COLUMNS], dtype='float64')

## 정규화

In [72]:
scaler = StandardScaler()
train_x_continue = scaler.fit_transform(train_x_continue)
test_x_continue = scaler.transform(test_x_continue)
val_x_continue = scaler.transform(val_x_continue)

* 정규화 내용 확인

In [73]:
print(train_x_continue[0].sum())
print(train_x_continue[1].sum())
print(train_x_continue[2].sum())
print(train_x_continue[3].sum())
print(train_x_continue[4].sum())

-4.1421630839022345
-1.8276499320635635
-3.8417757572076456
-3.840515541461756
-4.559329867801354


 - 정규화 한다고 다 더해서 1이 되거나 하지는 않는듯
 - 그렇다고 개별 값이 0~1은 아님

## Polynomial 하게 바꿔줌 
### (비선형적인 설정으로 선형 회귀를 확장하는 방법. 즉 다항식 함수로 바꿔줌)
    - 카테고리 값을 Polynomial로 바꿔줌

* sklearn.preprocessing.PolynomialFeatures 메소드
    - degree : 다항식 차수
    - interaction_only
        - default는 False
        - ex) degree = 3일 때, interaction_only=false 이면
            - a^2, a^3, b^2, b^3, ab, a^2*b, ab^2 Feature가 추가되고,
        - interaction_only=True 이면
            - ab만 추가됨
        

In [74]:
poly = PolynomialFeatures(degree=2, interaction_only=True)

In [75]:
train_x_category_poly = poly.fit_transform(train_x_continue)
test_x_category_poly = poly.fit_transform(test_x_continue)
val_x_category_poly = poly.fit_transform(val_x_continue)

In [76]:
train_x_category_poly.shape

(105246, 37)

 * np.unique : np.arr 내 중복 제거

In [77]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.keras.layers.advanced_activations import ReLU, PReLU, LeakyReLU, ELU
from tensorflow.python.keras.optimizers import Adam, SGD
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.models import Model
import tensorflow.keras.backend as K

In [78]:
def get_deep_model():
    
    category_inputs = []
    category_embeds = []
    
    # Categorical Data Embedding
    for i in range(len(CATEGORICAL_COLUMNS)):
        
        # input - embedding - flatten 순으로 layer 쌓기
        input_i = Input(shape=(1,), dtype='int32')
        
        dim = len(np.unique(data[CATEGORICAL_COLUMNS[i]]))
        # dim : data에서 카테고리별 요소가 몇 종류인지?
        
        embed_dim = int(np.ceil(dim ** 0.5))
        # embedding 차원을 0.5배 정도로 수행? (연산은 루트 올림인디?)
        # 왜 임베딩 차원을 이렇게 하는지 추후 검토
        
        embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
        # dim : 데이터가 몇 종류 있는지 = 임베딩 벡터를 몇 개 뽑아낼 것인지
        # embed_dim : 임베딩 처리 후 벡터의 차원 = 임베딩 벡터를 몇 차원 벡터로 뽑아 낼 것인지
        # input_length : 입력 데이터 길이
        
        flatten_i = Flatten()(embed_i)
        # category 값을 임베딩환 벡터들을 flatten
        # 서로 다른 값의 벡터 요소가 합쳐져도 괜찮은지??
        # 어차피 class에 대한 순서가 있으니 상관 없을듯
        
        category_inputs.append(input_i)
        category_embeds.append(flatten_i)
        
    # continuous 데이터 input
    continue_input = Input(shape=(len(CONTINUOUS_COLUMNS),))
    continue_dense = Dense(256, use_bias=False)(continue_input)
    # use_bias = False로 하는 이유는??
    
    # category와 continue를 합침
    concat_embeds = concatenate([continue_dense] + category_embeds)
    concat_embeds = Activation('relu')(concat_embeds)
    # Activation 효과 다시 공부
    # relu 말고 다른 것은 어떤지??
    bn_concat = BatchNormalization()(concat_embeds)
    # Batch Normalization 효과 다시 공부
    
    fc1 = Dense(512, use_bias=False)(bn_concat)
    relu1 = ReLU()(fc1)
    bn1 = BatchNormalization()(relu1)
    fc2 = Dense(256, use_bias=False)(bn1)
    relu2 = ReLU()(fc2)
    bn2 = BatchNormalization()(relu2)
    fc3 = Dense(128)(bn2)
    relu3 = ReLU()(fc3)
    
    return category_inputs, continue_input, relu3

In [79]:
def get_wide_model(poly):
    dim = poly.shape[1]
    return tf.keras.layers.Input(shape=(dim,))

# x_train_category_poly : 카테고리 데이터를 숫자로 바꾸고, Poly Feature를 추가한 것
# Poly Feature : a, b, c Feature를 이용해서 ab, bc, ca Feature를 만든것
# 데이터의 shape 만 가져옴

In [80]:
def get_embed_model(embed):
    dim = embed.shape[1]
    return tf.keras.layers.Input(shape=(dim,))

    * input - embedding - flatten 순으로 layer 쌓기

In [81]:
category_inputs, continue_input, deep_model = get_deep_model()
wide_model = get_wide_model(train_x_category_poly)
embed_model = get_embed_model(train_x_embedding)

### Wide모델과 Deep model을 합치기

* wide model, deep model 합치기

In [82]:
out_layer = concatenate([deep_model, wide_model, embed_model])

In [83]:
inputs = [continue_input] + category_inputs + [wide_model] + [embed_model]

In [84]:
output = Dense(len(label_key), activation='sigmoid')(out_layer)

In [85]:
model = Model(inputs=inputs, outputs=output)

In [86]:
len(inputs)

28

### 입력 데이터

    * 위에서 정의한 리스트 변수 inputs에 맞추어
    * continue 데이터 => category 데이터 => poly data 순으로 입력 값을 넣어준다

In [87]:
input_data = [train_x_continue] + [train_x_category[:, i] for i in range(train_x_category.shape[1])] + [train_x_category_poly] + [train_x_embedding]

In [88]:
val_data = [val_x_continue] + [val_x_category[:, i] for i in range(val_x_category.shape[1])] + [val_x_category_poly] + [val_x_embedding]

## 모델 구현

모델 구현에 대한 간단한 설명을 하겠습니다.  
입력은 2개로 분리해서 생각하면 됩니다.  

Wide 모델의 입력: category Feature을 Polynomial하게 바꿔준 데이터  
Deep 모델의 입력: category Feature을 embeding 시켜준 데이터 + continuous한 데이터  

그리고 출력은 lgbm 모델과 마찬가지로 1058개의 prediction 값이 row만큼 출력 됩니다.  

In [89]:
def binary_accuracy(y_true, y_pred):
    true = K.equal(y_true, 1.0 ) 
    pred = K.greater(y_pred , 0.5)
    true2 = K.cast(true , dtype = float)
    pred2 = K.cast(pred , dtype = float)
    return  K.sum(true2 * pred2) / K.sum(true2) 

gamma = 2.0
epsilon = K.epsilon()

def focal_loss(y_true, y_pred):
    # https://www.kaggle.com/mathormad/resnet50-v2-keras-focal-loss-mix-up
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = K.pow(1-pt, gamma) * CE
    loss = K.sum(FL, axis=1)
    return loss
    return K.mean(K.sum(loss, axis=1))

model.compile(optimizer='adam',
              loss=focal_loss   , # focal_loss  ,  # 'binary_crossentropy',
              metrics=[ binary_accuracy ]) 

In [90]:
checkpoint_path = "/ckpt/my_checkpoint/KM-{epoch:04d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 save_best_only = True , 
                                                 save_freq = 'epoch' , 
                                                 verbose=1)
# https://www.kaggle.com/rejpalcz/focalloss-for-keras
def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return tf.keras.callbacks.LearningRateScheduler(schedule)

lr_sched = step_decay_schedule(initial_lr=1e-4, decay_factor=0.75, step_size=2)

  # `val_loss`가 2번의 에포크에 걸쳐 향상되지 않으면 훈련을 멈춥니다.
Early = tf.keras.callbacks.EarlyStopping(min_delta=0.0001, 
                                         patience=10 ,
                                         monitor='val_loss')

In [91]:
epochs = 1000
batch_size = 32

## 임베딩

In [92]:
model.fit(input_data, train_y, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_data=(val_data, val_y), 
          callbacks=[lr_sched, Early])

Epoch 1/1000
3289/3289 [==============================] - 14s 4ms/step - loss: 4.3020 - binary_accuracy: 0.0125 - val_loss: 1.8203 - val_binary_accuracy: 4.2691e-04
Epoch 2/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.7199 - binary_accuracy: 0.0012 - val_loss: 1.6852 - val_binary_accuracy: 4.2691e-04
Epoch 3/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.6064 - binary_accuracy: 0.0019 - val_loss: 1.6400 - val_binary_accuracy: 4.2691e-04
Epoch 4/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.5613 - binary_accuracy: 0.0025 - val_loss: 1.6086 - val_binary_accuracy: 3.4153e-04
Epoch 5/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.5227 - binary_accuracy: 0.0028 - val_loss: 1.6079 - val_binary_accuracy: 5.1230e-04
Epoch 6/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.5048 - binary_accuracy: 0.0035 - val_loss: 1.5908 - val_binary_accuracy: 8.5383e-04
Epoch 7/10

## 새로운

In [63]:
model.fit(input_data, train_y, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_data=(val_data, val_y), 
          callbacks=[lr_sched, Early])

Epoch 1/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 4.1335 - binary_accuracy: 0.0112 - val_loss: 1.7965 - val_binary_accuracy: 4.2691e-04
Epoch 2/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.7538 - binary_accuracy: 6.5559e-04 - val_loss: 1.6905 - val_binary_accuracy: 6.8306e-04
Epoch 3/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.6523 - binary_accuracy: 9.3113e-04 - val_loss: 1.6490 - val_binary_accuracy: 2.5615e-04
Epoch 4/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.6094 - binary_accuracy: 0.0012 - val_loss: 1.6451 - val_binary_accuracy: 8.5383e-05
Epoch 5/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.5734 - binary_accuracy: 0.0014 - val_loss: 1.6351 - val_binary_accuracy: 4.2691e-04
Epoch 6/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.5498 - binary_accuracy: 0.0019 - val_loss: 1.6255 - val_binary_accuracy: 5.1230e-04
Ep

## 기존

In [35]:
model.fit(input_data, train_y, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_data=(val_data, val_y), 
          callbacks=[lr_sched, Early])

Epoch 1/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 3.7797 - binary_accuracy: 0.0134 - val_loss: 1.6372 - val_binary_accuracy: 0.0000e+00
Epoch 2/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.6078 - binary_accuracy: 2.7554e-04 - val_loss: 1.5695 - val_binary_accuracy: 0.0000e+00
Epoch 3/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.5426 - binary_accuracy: 5.9859e-04 - val_loss: 1.5531 - val_binary_accuracy: 0.0000e+00
Epoch 4/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.5162 - binary_accuracy: 0.0012 - val_loss: 1.5525 - val_binary_accuracy: 0.0000e+00
Epoch 5/1000
3289/3289 [==============================] - 13s 4ms/step - loss: 1.4884 - binary_accuracy: 0.0025 - val_loss: 1.5560 - val_binary_accuracy: 1.7077e-04
Epoch 6/1000
3289/3289 [==============================] - 12s 4ms/step - loss: 1.4717 - binary_accuracy: 0.0030 - val_loss: 1.5591 - val_binary_accuracy: 1.7077e-04
Ep

Wide & Deep 모델에서는 구매 품목의 imbalance 문제를 해결하기 위해서 focal loss를 도입했습니다.  
focal loss는 많은 비중을 차지해서 비교적 잘 분류되는 품목에는 영향력을 줄여주어서 분류가 잘 되지 않는 품목에 집중할 수 있도록 도와줍니다.  
직접 아웃풋을 분석한 결과 Focal loss를 사용하기 전에는 구매율이 높은 품목들만 추천했다면, 도입한 뒤로는 좀 더 다양한 추천을 해주는 것 같습니다. 하지만 아직 정확도가 낮은 문제점이 있습니다. 

In [36]:
# train input data와 같은 방식으로 test data를 input 형식에 맞추어줌
eval_input_data = [test_x_continue] + [test_x_category[:, i] for i in range(test_x_category.shape[1])] + [test_x_category_poly]

In [37]:
loss, acc = model.evaluate(eval_input_data, test_y)

193/193 [==============================] - 0s 2ms/step - loss: 1.6000 - binary_accuracy: 1.6192e-04


In [38]:
print(f'test_loss: {loss} - test_acc: {acc}')
# 문자열 앞에 f는 formating이었나? 확인

test_loss: 1.6000030040740967 - test_acc: 0.00016191709437407553


In [39]:
predict_y = model.predict(eval_input_data)

### 정확도 평가

In [40]:
''' 
pred_matrix: 예측된 아이템 행렬 파라미터.
top_n: 상위 몇개를 추천으로 사용할 지 정하는 파라미터.
test_matix: 고객ID와 Target이 있는 행렬 파라미터
'''
def get_acc(score_matrix, top_n, test_matix):
    avg_acc = 0
    for i in range(len(score_matrix)):
        top = score_matrix.iloc[i].nlargest(top_n).index
        tmp = 0
        for j in range(len(top)):
            if top[j] == test_matix["target"][i]:
                tmp += 1
        acc = tmp / len(top)
        avg_acc += acc / len(score_matrix)

    return avg_acc

In [41]:
dic = {'clnt_id': test_x['clnt_id'],'target' : test_x['clac_nm2']}
target_matrix = pd.DataFrame(dic).reset_index()
target_matrix = target_matrix.drop(['index'], axis=1)

target_matrix.head()

KeyError: 'clnt_id'

In [111]:
small_p = pd.DataFrame(predict_y[:1000])
small_p.head()

,0,1,2,3,4,5,6,7,8,9,...,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054
0,0.003870,0.009187,0.013846,0.001066,0.050542,0.012555,0.017187,0.040968,0.066419,0.032210,...,0.001556,0.003507,0.001765,0.006174,0.009276,0.002257,0.054420,0.004990,0.016043,0.002339
1,0.015519,0.007073,0.029056,0.004541,0.086528,0.004747,0.049089,0.037475,0.037094,0.049752,...,0.002761,0.001996,0.004467,0.002363,0.027529,0.002927,0.103238,0.018922,0.008862,0.002350
2,0.001252,0.048368,0.045424,0.000945,0.090145,0.016341,0.086308,0.094145,0.137432,0.054816,...,0.001171,0.003011,0.005142,0.001550,0.045851,0.002800,0.096974,0.003262,0.068804,0.004877
3,0.022368,0.008663,0.030572,0.006060,0.073585,0.004553,0.042916,0.029611,0.041933,0.058360,...,0.001998,0.001469,0.003201,0.002157,0.029728,0.002877,0.108355,0.019072,0.006630,0.002142
4,0.060900,0.062453,0.007792,0.026579,0.022882,0.006127,0.009927,0.007433,0.020697,0.028880,...,0.004389,0.015503,0.005152,0.001835,0.021648,0.001776,0.030017,0.009923,0.079559,0.000965


In [112]:
accuracy  = get_acc(small_p, 5, target_matrix)

print(f"정확도: {accuracy*100}%")

정확도: 2.0999999999999983%


### 정확도 평가

In [113]:
def get_pred_list(predict_y, top_n, target_matrix, columns):
    test_matrix = target_matrix.copy()
    pred_matrix = predict_y.copy()
    pred_matrix.rename(columns = columns['hangle'], inplace = True)
    for i in range(len(pred_matrix)):
        top = pred_matrix.iloc[i].nlargest(top_n).index
        top = pd.DataFrame(top.astype(str).to_frame().apply(lambda x: ", ".join(x)))
        test_matrix.loc[i, 'pred'] = top.values
    test_matrix['target'] = test_matrix['target'].apply(lambda x: columns['hangle'][x])
    return test_matrix

In [141]:
d={'hangle': Input['clac_nm2'], 'label': data['clac_nm2']}
df = pd.DataFrame(data=d).drop_duplicates()
cate2papago = df.set_index('label').to_dict()

In [142]:
test = get_pred_list(small_p, 5, target_matrix, cate2papago)

In [143]:
test[test['pred'].notna()]

,clnt_id,target,pred
0,42301,여자 로퍼,"[여자 골프 의류 세트, 여성 가죽 의류, 키즈 우산, 스포츠 가방, 치즈'!]"
1,29233,제너럴 요구르트,"[즉석 죽, 포도, 국내 Beefs-Rounds, 옥수수 스낵, 우유]"
2,58449,밤,"[포도, 아기 매트리스 패드, 냉동 떡볶이, 냉동 튀김 식품, 라면]"
3,48924,밤,"[옥수수 스낵, 즉석 죽, 제너럴 티 드링크, 국내 Beefs-Rounds, 포도]"
4,47681,여자 로퍼,"[여자 로퍼, 여성 청바지, 펫 도그 푸드, 조리 기구 세트, 여성 가죽 의류]"
...,...,...,...
995,16419,여성 스웨터 / 풀오버,"[남성 정장, 여성 청바지, 기타 컴퓨터 액세서리, 다른 영양학적 Supplemen..."
996,70886,팬케이크 믹스,"[옥수수 스낵, 국내 Beefs-Rounds, 라면, 즉석 죽, 제너럴 티 드링크]"
997,16419,여성 스웨터 / 풀오버,"[남성 정장, 여성 청바지, 기타 컴퓨터 액세서리, 다른 영양학적 Supplemen..."
998,47330,기능성 우유,"[즉석 죽, 옥수수 스낵, 포도, 일반 스낵, 라면]"
